#### Hessenberg decomposition

A Hessenberg matrix is a special kind of square matrix, one that is "almost" triangular

To be exact, an upper Hessenberg matrix has zero entries below the first subdiagonal, and a lower Hessenberg matrix has zero entries above the first superdiagonal

Similar to QR, Hessenberg decomposition aims to find for $A\in \mathbf{R}^{m \times m}$, orthogonal matrix $Q$ such that

$$A=QHQ^T$$

where $H$ is in Hessenberg form

#### Arnoldi iteration

The Arnoldi iteration performs exactly like modified Gram-Schmidt for progressively finding both first $(n+1)$ vectors in $Q$ and the $(n+1) \times n$ upper-left section of $H$

To see this, we can write more explicitly a `reduced version` of $AQ=QH$

$$A\begin{bmatrix}q_1 & q_2 & \cdots q_n\end{bmatrix}=\begin{bmatrix}q_1 & q_2 & \cdots q_{n+1}\end{bmatrix}\begin{bmatrix}h_{11} & \cdots & h_{1n} \\ h_{21}& & \vdots \\
& \ddots & \vdots \\ & & h_{n+1,n}\end{bmatrix}$$

The reason we have $n+1$ on the right is that for each $Aq_n$, the computation requires $q_{n+1}$ and $h_{n+1,n}$ due to the non-zeros in the first subdiagonal (We can see this from the full $AQ=QH$)

Different from MGS where for each $a_n$ only one new $q_n$ is produced, for Arnoldi, we will use the residual to compute $q_{n+1}$ and $h_{n+1, n}$

#### Example

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def arnoldi_iteration(A, b, n, tol=1e-10):
    m = A.shape[0]
    Q = np.zeros((m, n+1))
    H = np.zeros((n+1, n))
    Q[:, 0] = b / np.linalg.norm(b)

    for i in range(n):
        v = A @ Q[:, i]

        for j in range(i+1):
            H[j, i] = Q[:, j] @ v
            v -= H[j, i] * Q[:, j]

        H[i+1, i] = np.linalg.norm(v)
        if H[i+1, i] < tol:
            break
        Q[:, i+1] = v / H[i+1, i]

    return Q, H

In [ ]:
np.random.seed(42)

m = 20
A = np.random.rand(m, m)
b = np.random.rand(m)

Q, H = arnoldi_iteration(A, b, 8)

In [ ]:
print('Q:\n', Q)
print('\nCheck orthogonality in Q:\n', Q.T @ Q)
print('\nH:\n', H)

print('\nAQ:\n', A @ Q[:,:-1])
print('\nQH:\n', Q @ H)
print('\nDifference:\n', np.linalg.norm(A @ Q[:,:-1] - Q @ H))

Q:
 [[ 0.0372  0.3754 -0.0822 -0.1269 -0.1374  0.3090 -0.1037  0.2582 -0.0812]
 [ 0.3253 -0.1501 -0.1283  0.0422  0.1521 -0.2627  0.0749 -0.1871  0.0420]
 [ 0.1821  0.1901 -0.3645  0.1837  0.0338 -0.2874 -0.2249  0.1245  0.0999]
 [ 0.2979 -0.1769  0.4209 -0.1180  0.2731 -0.0193 -0.0526 -0.0868  0.0379]
 [ 0.1154  0.2312  0.0706 -0.0581 -0.2707 -0.3097 -0.2874  0.1174  0.1917]
 [ 0.3228 -0.0825 -0.1709  0.3594 -0.2822  0.1634 -0.2072 -0.3249 -0.0789]
 [ 0.1403  0.1458  0.3360  0.1391 -0.0221 -0.1200  0.0776  0.1686 -0.4739]
 [ 0.0039  0.3253  0.5194  0.1229  0.1330 -0.2568  0.0800  0.0791 -0.0813]
 [ 0.3263 -0.1678 -0.1651 -0.0117  0.1403 -0.1632  0.5055  0.0005 -0.1431]
 [ 0.0329  0.5119 -0.0879  0.0376  0.0403  0.3302  0.2378 -0.1666 -0.0242]
 [ 0.1151  0.1933  0.0316  0.4240 -0.2864 -0.0658  0.2793  0.0866 -0.0308]
 [ 0.3424 -0.1637 -0.1015  0.0060  0.0068  0.0441  0.0832  0.6425  0.2130]
 [ 0.3426 -0.0580  0.0540 -0.4171 -0.3111  0.1250 -0.0923  0.2139 -0.1795]
 [ 0.2067  0.2557 -0.

#### Relation to Krylov subspace

For $n=1$, we know that $q_1=\frac{b}{\|b\|}\in K_1(A, b)=\text{span}(b)$

Now assume for certain $n$, $q_1, \cdots, q_n \in K_n(A,b)$, when we compute $v=Aq_n$, we know that by definition

$$v\in AK_n(A,b) \subseteq K_{n+1}(A,b) $$

To get $q_{n+1}$, we need to subtract projections of $v$ onto previous $q_j, j=1, \cdots, n$

$$v=Aq_n-\sum_{j=1}^nh_{jn}q_j, \,\,q_{n+1}=\frac{v}{h_{n+1,n}}$$

since each $q_j \in K_n(A,b)\subseteq K_{n+1}(A,b)$, we know $q_{n+1}\in K_{n+1}(A,b)$, as it is the linear combination of vectors in it

Therefore, the orthonormal vectors from Arnoldi algorithm satisfy

$$\text{span}(q_1, q_2, \cdots, q_n)=K_n(A, b)=\text{span}(b, Ab, \cdots, A^{n-1}b)$$

and thus form `orthonomal basis` for Krylov subspaces